# Palmer Penguins

This notebook is inspired by an example [Radar chart with ggradar](https://r-graph-gallery.com/web-radar-chart-with-R.html).

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
fun rescaleInGroupDataFrame(df: DataFrame<*>, valueCol: String, groupCol: String, rescaledCol: String? = null): DataFrame<*> {
    val rescaledColName = if (rescaledCol == null) {
        "rescaled_${valueCol}"
    } else {
        rescaledCol
    }
    fun rescaleSubDataFrame(subDf: DataFrame<*>): DataFrame<*> {
        val minValue = subDf.minByOrNull(valueCol)?.let { it[valueCol] } as Double
        val maxValue = subDf.maxByOrNull(valueCol)?.let { it[valueCol] } as Double
        return subDf.add(rescaledColName) { (valueCol<Double>() - minValue) / (maxValue - minValue) }
    }
    return df.select(groupCol).distinct().map { v -> rescaleSubDataFrame(df.filter { groupCol<String>() == v[groupCol] }) }.concat()
}

fun getData(): DataFrame<*> {
    val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/penguins.csv")
        .dropNulls()
        .rename("bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g")
            .into("avg. bill length", "avg. bill depth", "avg. flipper length", "avg. body mass")
        .groupBy("species")
        .mean()
        .gather("avg. bill length", "avg. bill depth", "avg. flipper length", "avg. body mass")
            .into("variable", "value")
        .update { "value"<Double>() }
            .where { "variable"<String>() == "avg. body mass" }
            .with { it / 1000 }
        .add("units") {
            when ("variable"<String>()) {
                "avg. body mass" -> "kg"
                else -> "mm"
            }
        }
    return rescaleInGroupDataFrame(df, "value", "variable")
        .add("rescaled_value_pct") { floor(100 * "rescaled_value"<Double>()) }
        .convert { "rescaled_value_pct"<Double>() }.to<Int>()
        .sortBy("species")
}

In [3]:
val df = getData()
df.head(4)

species,variable,value,units,rescaled_value,rescaled_value_pct
Adelie,avg. bill length,38.823973,mm,0.000000,0
Adelie,avg. bill depth,18.347260,mm,0.978584,97
Adelie,avg. flipper length,190.102740,mm,0.000000,0
Adelie,avg. body mass,3.706164,kg,0.000000,0


In [4]:
val fontFamily = "roboto"
val axisColor = "lightgray"
val axisTextData = mapOf(
    "x" to List(3) { "avg. bill length" },
    "y" to listOf(0, 50, 100),
    "text" to listOf("0%", "50%", "100%")
)
val penguinsColors = mapOf(
    "Adelie" to "#ff5a5f",
    "Chinstrap" to "#ffb400",
    "Gentoo" to "#007a87"
)
val penguinsTooltips = layerTooltips().title("@species").line("@variable (@units): @value").format("@value", ".2~f")
val penguinsTheme = theme(
    text = elementText(family = fontFamily, size = 18),
    plotTitle = elementText(size = 28, hjust = .5, face = "bold"),
    axisTitle = "blank", axisTextY = "blank", axisLineX = "blank",
    axisTicks = elementLine(color = axisColor),
    panelGrid = elementLine(color = axisColor),
    panelInset = Pair(0, 100),
    tooltip = elementRect(),
    axisTooltip = "blank",
).legendPosition(1, 0).legendJustification(1, 0)

In [5]:
letsPlot(df.toMap()) +
    geomArea(position = positionIdentity, flat = true,
             size = 2.5, colorBy = "paint_a", fillBy = "paint_a", alpha = .2)
        { x = "variable"; y = "rescaled_value_pct"; paint_a = "species" } +
    geomPoint(size = 6, colorBy = "paint_a", tooltips = penguinsTooltips)
        { x = "variable"; y = "rescaled_value_pct"; paint_a = "species" } +
    geomText(data = axisTextData, hjust = 1, fontface = "bold", family = fontFamily, size = 10)
        { x = "x"; y = "y"; label = "text" } +
    scaleXDiscrete() +
    scaleManual("paint_a", name = "", values = penguinsColors) +
    coordPolar(ylim = Pair(-15, 100)) +
    ggsize(800, 600) +
    ggtitle("Penguins species") +
    penguinsTheme + flavorSolarizedLight()

<path d="M263.5 235.09540064678407 L269.041462434558 235.64118705861466 L274.3699695754439 237.25757202838042 L279.2807498814946 239.8824387881724 L283.585084819546 243.414915180454 L287.1175612118276 247.71925011850539 L289.7424279716196 252.63003042455608 L291.35881294138534 257.958537565442 L291.90459935321593 263.5 L291.35881294138534 269.041462434558 L289.7424279716196 274.36996957544386 L287.1175612118276 279.2807498814946 L283.585084819546 283.585084819546 L279.2807498814946 287.1175612118276 L274.3699695754439 289.7424279716196 L269.041462434558 291.35881294138534 L263.5 291.904599353216 L257.958537565442 291.35881294138534 L252.6300304245561 289.7424279716196 L247.71925011850539 287.1175612118276 L243.414915180454 283.585084819546 L239.8824387881724 279.2807498814946 L237.25757202838045 274.3699695754439 L235.64118705861466 269.041462434558 L235.09540064678404 263.5 L235.64118705861466 257.958537565442 L237.25757202838045 252.63003042455608 L239.8824387881724 247.71925011850539 L243.414915180454 243.414915180454 L247.71925011850539 239.8824387881724 L252.63003042455608 237.25757202838048 L257.958537565442 235.64118705861466 L263.5 235.09540064678407 " stroke="rgb(192,192,192)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M263.5 140.41340280273084 L275.56459627017284 141.0060984112249 L287.5130038830847 142.77847725399693 L299.2301531417593 145.7134703539101 L310.6032014935902 149.7828121229819 L321.52262026580865 154.9473125754477 L331.88324948647676 161.1572347487471 L341.5853106325033 168.35277369766243 L350.5353675513661 176.4646324486339 L358.6472263023376 185.4146893674967 L365.8427652512529 195.11675051352324 L372.0526874245523 205.47737973419135 L377.2171878770181 216.39679850640982 L381.2865296460899 227.76984685824073 L384.2215227460031 239.4869961169153 L385.9939015887751 251.43540372982716 L386.58659719726916 263.5 L385.9939015887751 275.56459627017284 L384.2215227460031 287.5130038830847 L381.2865296460899 299.2301531417593 L377.2171878770181 310.6032014935902 L372.0526874245523 321.52262026580865 L365.8427652512529 331.8832494864767 L358.6472263023376 341.5853106325033 L350.5353675513661 350.5353675513661 L341.5853106325033 358.6472263023376 L331.8832494864767 365.8427652512529 L321.52262026580865 372.0526874245523 L310.6032014935902 377.2171878770181 L299.2301531417593 381.2865296460899 L287.5130038830847 384.2215227460031 L275.56459627017284 385.9939015887751 L263.5 386.58659719726916 L251.43540372982716 385.9939015887751 L239.48699611691526 384.2215227460031 L227.76984685824075 381.2865296460899 L216.39679850640982 377.2171878770181 L205.47737973419135 372.0526874245523 L195.1167505135233 365.84276525125296 L185.4146893674967 358.64722630233763 L176.4646324486339 350.5353675513661 L168.35277369766243 341.58531063250337 L161.15723474874707 331.88324948647676 L154.9473125754477 321.52262026580865 L149.78281212298197 310.60320149359023 L145.7134703539101 299.2301531417593 L142.77847725399693 287.5130038830847 L141.0060984112249 275.56459627017284 L140.41340280273087 263.5 L141.0060984112249 251.43540372982721 L142.77847725399693 239.4869961169153 L145.7134703539101 227.76984685824073 L149.78281212298197 216.39679850640977 L154.9473125754477 205.47737973419135 L161.15723474874704 195.1167505135233 L168.35277369766243 185.41468936749663 L176.4646324486339 176.4646324486339 L185.41468936749663 168.35277369766243 L195.11675051352327 161.1572347487471 L205.47737973419132 154.9473125754477 L216.39679850640974 149.78281212298197 L227.76984685824073 145.7134703539101 L239.48699611691524 142.77847725399693 L251.4354037298272 141.0060984112249 L263.5 140.41340280273084 " stroke="rgb(192,192,192)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M263.5 45.73140495867767 L274.185398500227 45.99371676854304 L284.84505493953657 46.780020266013366 L295.45328927189075 48.08842117709105 L305.9845453316114 49.91576744937913 L316.41345240042153 52.25765684565101 L326.714886327728 55.10844